### Código completo para análise de dados de temperatura de um secador experimental. Aqui estão as principais características:

#### Funcionalidades Implementadas:
###### 1.Simulação Realística de Dados 🔥

25 medições em 6 horas (a cada 15 minutos)
Comportamento realístico: aquecimento inicial + operação estável com flutuações
Ruído aleatório simulando variações reais do equipamento

###### 2.Armazenamento de Dados 📊

Lista: todas as temperaturas em sequência temporal
Dicionário: mapeamento horário → temperatura
DataFrame: para análises avançadas e exportação

###### 3.Cálculos Estatísticos 📈

Média, mediana, desvio padrão
Temperaturas mín/máx e amplitude
Coeficiente de variação (indicador de estabilidade)

###### 4.Visualizações Completas 🎯

Gráfico temporal: mostra a evolução da temperatura
Histograma: distribuição dos valores
Box plot: análise de dispersão e outliers
Gráfico por períodos: comparação entre diferentes fases

###### 5.Interpretação Técnica 🔍

Avaliação da estabilidade do processo
Análise da adequação da temperatura
Recomendações técnicas específicas

##### Como Usar:

Execute o script - ele simula automaticamente os dados
Visualize os gráficos gerados
Analise as estatísticas impressas no console
Leia a interpretação técnica dos resultados

Arquivo CSV exportado para análises futuras

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import random

# Configurações
plt.style.use('seaborn-v0_8')
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)

class SecadorExperimental:
    def __init__(self, temp_inicial=45, temp_final=75, duracao_horas=6, intervalo_min=15):
        """
        Inicializa o simulador do secador experimental
        
        Args:
            temp_inicial: Temperatura inicial em °C
            temp_final: Temperatura final de operação em °C
            duracao_horas: Duração total do experimento em horas
            intervalo_min: Intervalo entre medições em minutos
        """
        self.temp_inicial = temp_inicial
        self.temp_final = temp_final
        self.duracao_horas = duracao_horas
        self.intervalo_min = intervalo_min
        self.temperaturas = []
        self.horarios = []
        self.dados_dict = {}
    
    def simular_dados(self):
        """
        Simula dados realistas de temperatura para o secador experimental
        Considera aquecimento inicial, operação estável e pequenas flutuações
        """
        print("🔥 Simulando dados de temperatura do secador experimental...")
        
        # Número total de medições
        num_medicoes = int((self.duracao_horas * 60) / self.intervalo_min) + 1
        
        # Gerar horários
        inicio = datetime.now().replace(hour=8, minute=0, second=0, microsecond=0)
        for i in range(num_medicoes):
            horario = inicio + timedelta(minutes=i * self.intervalo_min)
            self.horarios.append(horario.strftime("%H:%M"))
        
        # Simular temperaturas com comportamento realístico
        for i in range(num_medicoes):
            tempo_norm = i / (num_medicoes - 1)  # Normalizado de 0 a 1
            
            if tempo_norm <= 0.3:  # Primeira fase: aquecimento (30% do tempo)
                # Aquecimento exponencial
                temp_base = self.temp_inicial + (self.temp_final - self.temp_inicial) * (1 - np.exp(-5 * tempo_norm / 0.3))
            else:  # Segunda fase: operação estável com flutuações
                temp_base = self.temp_final
                # Adicionar ciclos de variação térmica
                ciclo = np.sin(2 * np.pi * tempo_norm * 3) * 2  # Ciclos de ±2°C
            
            # Adicionar ruído aleatório (flutuações do equipamento)
            ruido = random.gauss(0, 1.5)  # Desvio padrão de 1.5°C
            
            if tempo_norm <= 0.3:
                temperatura = temp_base + ruido
            else:
                temperatura = temp_base + ciclo + ruido
            
            # Garantir limites físicos realísticos
            temperatura = max(20, min(85, temperatura))
            self.temperaturas.append(round(temperatura, 1))
        
        # Criar dicionário horários: temperaturas
        self.dados_dict = dict(zip(self.horarios, self.temperaturas))
        
        print(f"✅ Dados simulados: {len(self.temperaturas)} medições em {self.duracao_horas}h")
        return self.temperaturas
    
    def calcular_estatisticas(self):
        """
        Calcula estatísticas descritivas dos dados de temperatura
        """
        if not self.temperaturas:
            print("❌ Erro: Nenhum dado disponível. Execute simular_dados() primeiro.")
            return None
        
        stats = {
            'média': round(np.mean(self.temperaturas), 2),
            'mediana': round(np.median(self.temperaturas), 2),
            'desvio_padrão': round(np.std(self.temperaturas), 2),
            'temperatura_mín': round(min(self.temperaturas), 2),
            'temperatura_máx': round(max(self.temperaturas), 2),
            'amplitude': round(max(self.temperaturas) - min(self.temperaturas), 2),
            'coef_variação': round((np.std(self.temperaturas) / np.mean(self.temperaturas)) * 100, 2)
        }
        
        print("\n📊 ESTATÍSTICAS DE TEMPERATURA:")
        print("=" * 40)
        for chave, valor in stats.items():
            unidade = "%" if chave == "coef_variação" else "°C"
            print(f"{chave.replace('_', ' ').title()}: {valor} {unidade}")
        
        return stats
    
    def plotar_graficos(self):
        """
        Cria visualizações dos dados de temperatura
        """
        if not self.temperaturas:
            print("❌ Erro: Nenhum dado disponível.")
            return
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Análise de Temperatura do Secador Experimental', fontsize=16, fontweight='bold')
        
        # Gráfico 1: Variação temporal
        ax1.plot(range(len(self.temperaturas)), self.temperaturas, 'b-', linewidth=2, marker='o', markersize=4)
        ax1.axhline(y=np.mean(self.temperaturas), color='r', linestyle='--', label=f'Média: {np.mean(self.temperaturas):.1f}°C')
        ax1.fill_between(range(len(self.temperaturas)), self.temperaturas, alpha=0.3)
        ax1.set_title('Variação da Temperatura ao Longo do Tempo')
        ax1.set_xlabel('Medições (intervalos de 15 min)')
        ax1.set_ylabel('Temperatura (°C)')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Gráfico 2: Histograma
        ax2.hist(self.temperaturas, bins=15, color='skyblue', alpha=0.7, edgecolor='black')
        ax2.axvline(np.mean(self.temperaturas), color='red', linestyle='--', linewidth=2, label=f'Média: {np.mean(self.temperaturas):.1f}°C')
        ax2.set_title('Distribuição das Temperaturas')
        ax2.set_xlabel('Temperatura (°C)')
        ax2.set_ylabel('Frequência')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Gráfico 3: Box plot
        box_data = ax3.boxplot(self.temperaturas, patch_artist=True)
        box_data['boxes'][0].set_facecolor('lightgreen')
        ax3.set_title('Box Plot - Análise de Dispersão')
        ax3.set_ylabel('Temperatura (°C)')
        ax3.grid(True, alpha=0.3)
        
        # Gráfico 4: Variação por período
        periodos = ['0-1.5h', '1.5-3h', '3-4.5h', '4.5-6h']
        temps_por_periodo = []
        n_por_periodo = len(self.temperaturas) // 4
        
        for i in range(4):
            inicio = i * n_por_periodo
            fim = (i + 1) * n_por_periodo if i < 3 else len(self.temperaturas)
            temps_periodo = self.temperaturas[inicio:fim]
            temps_por_periodo.append(np.mean(temps_periodo))
        
        bars = ax4.bar(periodos, temps_por_periodo, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
        ax4.set_title('Temperatura Média por Período')
        ax4.set_ylabel('Temperatura Média (°C)')
        ax4.set_ylim(0, max(temps_por_periodo) * 1.1)
        
        # Adicionar valores nas barras
        for bar, temp in zip(bars, temps_por_periodo):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{temp:.1f}°C', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    
    def interpretar_resultados(self):
        """
        Fornece interpretação técnica dos resultados
        """
        if not self.temperaturas:
            return
        
        media = np.mean(self.temperaturas)
        cv = (np.std(self.temperaturas) / media) * 100
        
        print("\n🔍 INTERPRETAÇÃO DOS RESULTADOS:")
        print("=" * 50)
        
        # Análise da estabilidade
        if cv < 5:
            estabilidade = "EXCELENTE"
            cor_estab = "🟢"
        elif cv < 10:
            estabilidade = "BOA"
            cor_estab = "🟡"
        else:
            estabilidade = "NECESSITA AJUSTES"
            cor_estab = "🔴"
        
        print(f"{cor_estab} Estabilidade do processo: {estabilidade} (CV = {cv:.1f}%)")
        
        # Análise da faixa de operação
        if 60 <= media <= 80:
            print("🟢 Temperatura média adequada para secagem de produtos agrícolas")
        elif 50 <= media < 60:
            print("🟡 Temperatura um pouco baixa - pode aumentar tempo de secagem")
        else:
            print("🔴 Temperatura fora da faixa ideal - verificar calibração")
        
        # Recomendações
        print("\n💡 RECOMENDAÇÕES:")
        if cv > 10:
            print("• Investigar oscilações de temperatura - verificar controlador PID")
            print("• Avaliar isolamento térmico do equipamento")
        
        if max(self.temperaturas) - min(self.temperaturas) > 15:
            print("• Grande amplitude térmica detectada - verificar sensores")
        
        print("• Monitorar consumo energético em relação à estabilidade térmica")
        print("• Considerar implementar controle adaptativo para maior eficiência")
    
    def exportar_dados(self, nome_arquivo="dados_secador.csv"):
        """
        Exporta os dados para arquivo CSV
        """
        df = pd.DataFrame({
            'Horário': self.horarios,
            'Temperatura_°C': self.temperaturas
        })
        
        df.to_csv(nome_arquivo, index=False, encoding='utf-8')
        print(f"\n💾 Dados exportados para '{nome_arquivo}'")
        return df

# Execução principal
def main():
    print("🌡️  SISTEMA DE ANÁLISE DO SECADOR EXPERIMENTAL")
    print("=" * 55)
    
    # Criar instância do secador
    secador = SecadorExperimental(
        temp_inicial=45,    # Temperatura inicial
        temp_final=70,      # Temperatura de operação
        duracao_horas=6,    # 6 horas de operação
        intervalo_min=15    # Medições a cada 15 minutos
    )
    
    # 1. Simular dados
    temperaturas = secador.simular_dados()
    print(f"📋 Lista de temperaturas: {len(temperaturas)} medições")
    print(f"    Primeiras 5: {[float(t) for t in temperaturas[:5]]}")
    print(f"    Últimas 5: {[float(t) for t in temperaturas[-5:]]}")
    
    # 2. Calcular estatísticas (média incluída)
    stats = secador.calcular_estatisticas()
    
    # 3. Mostrar dicionário horários:temperaturas
    print(f"\n📚 Dicionário criado com {len(secador.dados_dict)} entradas")
    print("    Primeiras 3 entradas:")
    for i, (hora, temp) in enumerate(list(secador.dados_dict.items())[:3]):
        print(f"    {hora}: {temp}°C")
    
    # 4. Plotar gráficos
    print("\n📈 Gerando visualizações...")
    secador.plotar_graficos()
    
    # 5. Interpretar resultados
    secador.interpretar_resultados()
    
    # Exportar dados
    df = secador.exportar_dados()
    
    print(f"\n✅ Análise concluída com sucesso!")
    print("   Verifique os gráficos gerados e o arquivo CSV exportado.")

if __name__ == "__main__":
    main()